In [1]:
import os
os.chdir("..")

In [3]:

import json
import numpy as np
import math
from typing import Dict, List, Tuple
import copy
from itertools import combinations
from collections import Counter

import logging
from datetime import timedelta
from config import *
from utils import *
from main import *
from GtConflictFinder import GtConflictFinder
from ConflictDetector import ConflictDetector, ConflictPredicator


import plotly.graph_objects as go
from plotly.subplots import make_subplots

root_folder = os.path.join(DATA_ROOT, REFIT_ROOT)

test_projects = [
    # "House1",
    "House3",
    "House4",
    "House8",
    "House9",
    "House15",
    # "House18",
    # "House20",
]
ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            "weatherDesc#CAT": {},
            WEEKDAY_CTX: {
                "range": (0, 6),
                "interval": 1,
            },
            # HOLIDAY_CTX: {
            #     "range": (0,1),
            #     "interval": 1,
            # }
        })
capacity = {
    "TV":1,
    "WashingMachine":1,
    "PC": 1,
}
BOOL_SIM = False
BOOL_UMASS= False
ccp_alpha = 6e-5
test_dates = generate_test_date(root_folder, test_projects, test_ratio = 0.4, true_random=False, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)
# test_dates = []


In [4]:
habit_groups = {}
for p in test_projects:
    habit_groups[p], grid_data = test_umass(root_folder, test_project=p, ctx_info=ctx_info, ccp_alpha=ccp_alpha, test_dates=test_dates, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)

In [5]:
#Exps:
#   1. Contrived environment with injected conflicts:
#   2. Real dataset with groundtruth conflicts:
#   3. observations about the algorithm: with different threshold, different capacity for the devices, different number of users.

In [6]:
for home, groups in habit_groups.items():
    print("The habit groups found in {}".format(home))
    for d, d_groups in groups.items():
        print(d + " " + str(len(d_groups)))
        # if d != "TV":
        #     continue
        # for g in d_groups:
        #     print(ctx_info.coor_box_to_range(g["box"][0] + g["box"][1]))
        #     print(g["dis"])


The habit groups found in House3
WashingMachine 14
TV 56
The habit groups found in House4
PC 6
WashingMachine 1
TV 51
The habit groups found in House8
PC 8
WashingMachine 12
TV 26
The habit groups found in House9
WashingMachine 3
TV 23
The habit groups found in House15
TV 44
WashingMachine 9


In [7]:
c_detector = ConflictDetector(ctx_info, capacity)
final_conflicts = c_detector.predict_conflict_scenarios(habit_groups)
print({x:len(final_conflicts[x]) for x in final_conflicts})

{'TV': 1035, 'WashingMachine': 182, 'PC': 15}


In [8]:
import plotly.express as px
# for x in final_conflicts["Window"]:
#     print((ctx_info.coor_box_to_range(x["box"]),x["prob"]))
probs_i = [(x["prob"], i) for i, x in enumerate(final_conflicts["TV"])]
# print(sorted(probs_i))
# fig = px.histogram(probs)
# fig.show()

In [9]:
import copy
# Make ground truth:

gt_ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            "weatherDesc#CAT": {},
            WEEKDAY_CTX: {
                "range": (0, 6),
                "interval": 1,
            },
            # HOLIDAY_CTX: {
            #     "range": (0,1),
            #     "interval": 1,
            # },
        })
device_events = {}
for p in test_projects:
    ctx_evts, device_evts = load_processed(root_folder, p, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)
    device_events[p] = device_evts

gtconflict_cfg = {
    "context_info": gt_ctx_info,
    "capacity": capacity
}
# test_dates = {}

conflict_finder = GtConflictFinder(gtconflict_cfg)
gt_conflicts, test_state_cnt = conflict_finder.get_Gt_conflict(ctx_evts, device_events, test_dates)
print(len(gt_conflicts))

TypeError: load_processed() missing 1 required positional argument: 'project'

In [ ]:

# TEST_RATIO = 0.3
# conflict_time = [x['cur_time'] for x in conflicts]
# end_time = max(conflict_time)
# start_time = min(conflict_time)
# print(start_time)
# print(end_time)
# total_time_range = end_time - start_time
# test_start = end_time - total_time_range * TEST_RATIO
# print(test_start)

conflict_device = {
    d:[]
    for d in capacity
}

conflict_state_device = {
    d:{}
    for d in capacity
}
MIN_TEST_OBS = 20
cnts = []
for c in gt_conflicts:
    d = c["device"]
    conflict_device[d].append(c)
    s = gt_ctx_info.get_coor_by_ctx(c["ctx"])
    users = frozenset(c["device_states"].keys())
    cnts.append(test_state_cnt[d][users][s])
    if test_state_cnt[d][users][s] < MIN_TEST_OBS:
        continue
    if s not in conflict_state_device[d]:
        conflict_state_device[d][s] = {}
    conflict_state_device[d][s][users] = conflict_state_device[d][s].get(users, 0) + 1

    # if conflict_state_device[d][s][users] > test_state_cnt[d][s]:
    #     print(conflict_state_device[d][s][users], test_state_cnt[d][s])
    #     print(users, d, c["cur_time"], s)
    
print({d:len(c) for d,c in conflict_device.items()})
print({d:len(c) for d,c in conflict_state_device.items()})
print(len(test_dates))
print(Counter(cnts))

{'TV': 6223, 'WashingMachine': 85, 'PC': 1}
{'TV': 24, 'WashingMachine': 12, 'PC': 0}
203
Counter({18.0: 1044, 22.0: 585, 12.0: 507, 8.0: 484, 11.0: 418, 9.0: 410, 10.0: 334, 16.0: 320, 7.0: 282, 2.0: 280, 21.0: 275, 14.0: 229, 13.0: 203, 5.0: 188, 6.0: 181, 1.0: 146, 3.0: 127, 17.0: 91, 15.0: 83, 24.0: 42, 4.0: 33, 26.0: 28, 20.0: 14, 19.0: 5})


In [29]:
all_users = test_projects
all_devices = capacity.keys()
user_pairs = list(combinations(all_users, 2))
conflict_predicator = ConflictPredicator(ctx_info, final_conflicts)
exp_cnt = 0
exp_gt_c = 0
exp_result = {d:[0,0,0,0,0] for d in all_devices}
max_p = 0
all_state_cnt = 0
for d in all_devices:
    for u_pair in user_pairs:
        u_pair_set = frozenset(u_pair)
        it = np.nditer(test_state_cnt[d][u_pair_set], flags=['multi_index'])
        for count in it:
            all_state_cnt += count
            if count < MIN_TEST_OBS:
                continue
            state = it.multi_index
            gt_prob = 0.
            if (d in conflict_state_device) and \
                (state in conflict_state_device[d]) and \
                (u_pair_set in conflict_state_device[d][state]):

                gt_prob =  float(conflict_state_device[d][state][u_pair_set]) / count
            gt_ctx_snapshot = gt_ctx_info.coor_to_snapshot(state)

            pred_prob = conflict_predicator.get_prob_conflict(ctx_info.get_coor_by_ctx(gt_ctx_snapshot), u_pair, d)
            exp_cnt += 1
            if gt_prob > 0:
                # if gt_prob > 0.5:
                #     print(gt_ctx_info.coor_to_snapshot(state).values(), gt_prob, pred_prob, u_pair)
                exp_gt_c += 1
                exp_result[d][0] += abs(pred_prob - gt_prob)
                exp_result[d][1] += 1
                exp_result[d][4] += gt_prob
                if gt_prob > 1:
                    print(gt_prob, pred_prob, d, u_pair)
                max_p = max(gt_prob, max_p)

            else:
                exp_result[d][2] += pred_prob - gt_prob
                exp_result[d][3] += 1

for d in exp_result:
    exp_result[d].append((exp_result[d][0] + exp_result[d][2]) / (exp_result[d][1] + exp_result[d][3]))
    if exp_result[d][1] > 0:
        exp_result[d][0] = exp_result[d][0] / exp_result[d][1]
        exp_result[d][4] = exp_result[d][4] / exp_result[d][1]
    if exp_result[d][3] > 0:
        exp_result[d][2] = exp_result[d][2] / exp_result[d][3]
    
print(max_p)
print(exp_result)
print(all_state_cnt)

0.8181818181818182
{'TV': [0.0971058248475422, 158, 0.00549340799066953, 92, 0.2698205275736919, 0.06339245544421306], 'WashingMachine': [0.042355998573244825, 13, 0.0010847283899275608, 237, 0.04472514664822357, 0.0032308344394600583], 'PC': [0, 0, 1.3873976113410812e-05, 250, 0, 1.3873976113410812e-05]}
14582.0


Run grid search for the optimal ccp_alpha

In [20]:
from refit_alpha_test import *

ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "weatherDesc#CAT": {},
            WEEKDAY_CTX: {
                "range": (0, 6),
                "interval": 1,
            },
            # HOLIDAY_CTX: {
            #     "range": (0,1),
            #     "interval": 1,
            # },
        })
gt_ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 60,
            },
            # "humidity#NUM" : {
            #     "range" : (0, 100),
            #     "interval" : 10,
            # },
            # "FeelsLikeC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            # "tempC#NUM" : {
            #     "range" : (-10., 40),
            #     "interval" : 5,
            # },
            "weatherDesc#CAT": {},
            WEEKDAY_CTX: {
                "range": (0, 6),
                "interval": 1,
            },
            # HOLIDAY_CTX: {
            #     "range": (0,1),
            #     "interval": 1,
            # },
        })


def alpha_nonLinear_generator(alpha_min, alpha_max, alpha_step):
    auc_values = []
    alpha = alpha_min
    while alpha <= alpha_max:
        yield alpha
        if alpha_step <= alpha / 10:
            alpha_step *= 10
        alpha += alpha_step

print("Run test accuracy for context " + str(ctx_info.get_all_ctx_ordered()))
alpha_generator = alpha_nonLinear_generator(1e-6, 0.002, 1e-6)
acc_values = test_acc_alpha(ctx_info, gt_ctx_info, alpha_generator)


Run test accuracy for context ['min_of_day#NUM']
Finish alpha 1e-06, result (0.09154646286912588, 0.011197153403552644, 0.06197791698579493)
Finish alpha 2e-06, result (0.09161423441758597, 0.011136424576801987, 0.06199840039617747)
Finish alpha 3e-06, result (0.09162504735961662, 0.011158099657084421, 0.062013210605084766)
Finish alpha 4e-06, result (0.09149464975236093, 0.011163378185982717, 0.061932741815933744)
Finish alpha 4.9999999999999996e-06, result (0.09149464975236093, 0.011174628597153414, 0.06193688196724457)
Finish alpha 5.999999999999999e-06, result (0.09149464975236093, 0.011115331505712325, 0.06191506063759424)
Finish alpha 6.999999999999999e-06, result (0.0916163206105823, 0.011111319215704815, 0.06199048009726739)
Finish alpha 8e-06, result (0.0916163206105823, 0.011111319215704815, 0.06199048009726739)
Finish alpha 9e-06, result (0.0916163206105823, 0.011111319215704815, 0.06199048009726739)
Finish alpha 1e-05, result (0.0916163206105823, 0.011111319215704815, 0.061